In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import random
import re

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [2]:
train_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/train_featured.csv')
test_data = pd.read_csv('~/Documents/Datos/DataSets/TP2/test_featured.csv')
test_data['clean_text'].fillna("", inplace=True)

In [11]:
train_data.location_original.value_counts()

USA                           104
New York                       71
United States                  50
London                         45
Canada                         29
                             ... 
West Vancouver, B.C.            1
?Gangsta OC / MV RP; 18+.?      1
Bolivar, MO                     1
Reading UK                      1
Photo : Blue Mountains          1
Name: location_original, Length: 3341, dtype: int64

In [19]:
location_dic = {}
for location in train_data.location_original:
    if (location != location): continue
    try:
        location_dic[location] += 1
    except KeyError:
        location_dic[location] = 1
    except TypeError:
        pass
location_dic

{'Birmingham': 4,
 'Est. September 2012 - Bristol': 1,
 'AFRICA': 1,
 'Philadelphia, PA': 4,
 'London, UK': 10,
 'Pretoria': 2,
 'World Wide!!': 1,
 'Paranaque City': 1,
 'Live On Webcam': 2,
 'milky way': 1,
 'GREENSBORO,NORTH CAROLINA': 1,
 'England.': 1,
 'Sheffield Township, Ohio': 1,
 'India': 24,
 'Barbados': 4,
 'Anaheim': 1,
 'Abuja': 1,
 'USA': 104,
 'South Africa': 8,
 'Sao Paulo, Brazil': 1,
 'hollywoodland ': 1,
 'Edmonton, Alberta - Treaty 6': 1,
 'Inang Pamantasan': 1,
 'Twitter Lockout in progress': 1,
 'Concord, CA': 1,
 'Calgary, AB': 5,
 'San Francisco': 14,
 'CLVLND': 1,
 'Nashville, TN': 9,
 'Santa Clara, CA': 1,
 'UK': 27,
 'St. Louis, MO': 3,
 'Walker County, Alabama': 1,
 'Australia': 18,
 'North Carolina': 5,
 'Norf Carolina': 1,
 'San Mateo County, CA': 1,
 'Njoro, Kenya': 1,
 "Your Sister's Bedroom": 1,
 'Arlington, TX': 3,
 'South Bloomfield, OH': 1,
 'New Hanover County, NC': 2,
 'Maldives': 2,
 'Manchester, NH': 2,
 'Wilmington, NC': 2,
 'global': 1,
 'Albe

In [20]:
location_dic = {k: v for k, v in sorted(location_dic.items(), key=lambda item: item[1], reverse=True)}
location_dic = {k: v for k, v in location_dic.items() if v >= 10}
location_dic

{'USA': 104,
 'New York': 71,
 'United States': 50,
 'London': 45,
 'Canada': 29,
 'Nigeria': 28,
 'UK': 27,
 'Los Angeles, CA': 26,
 'India': 24,
 'Mumbai': 22,
 'Washington, DC': 21,
 'Kenya': 20,
 'Worldwide': 19,
 'Australia': 18,
 'Chicago, IL': 18,
 'California': 17,
 'New York, NY': 15,
 'California, USA': 15,
 'Everywhere': 15,
 'San Francisco': 14,
 'Florida': 14,
 'United Kingdom': 14,
 'Indonesia': 13,
 'Los Angeles': 13,
 'Washington, D.C.': 13,
 'Toronto': 12,
 'NYC': 12,
 'Ireland': 12,
 'San Francisco, CA': 11,
 'Chicago': 11,
 'Earth': 11,
 'Seattle': 11,
 'London, UK': 10,
 'Texas': 10,
 'New York City': 10,
 'ss': 10,
 'Atlanta, GA': 10,
 'Sacramento, CA': 10,
 'London, England': 10}

In [22]:
for x in location_dic.keys():
    train_data[x] = 0
    test_data[x] = 0

In [23]:
def one_hot_hashtags(row):
    loc = row['location_original']
    try:
        if loc in row:
            row[loc] = 1
    except AttributeError:
        pass
    return row

In [24]:
train_data = train_data.apply(one_hot_hashtags, axis=1)
train_data.head()

,id_original,keyword_original,location_original,text_original,target_label,special_chars_count,hashtags,labels,hashtags_count,labels_count,...,Chicago,Earth,Seattle,"London, UK",Texas,New York City,ss,"Atlanta, GA","Sacramento, CA","London, England"
0,1,NaN,NaN,our deeds are the reason of this earthquake ma...,1,1,#earthquake,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
1,4,NaN,NaN,forest fire near la ronge sask canada,1,1,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,NaN,NaN,all residents asked to shelter in place are be...,1,3,NaN,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,NaN,NaN,people receive wildfires evacuation orders in ...,1,2,#wildfires,NaN,1,0,...,0,0,0,0,0,0,0,0,0,0
4,7,NaN,NaN,just got sent this photo from ruby alaska as s...,1,2,#alaska #wildfires,NaN,2,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
test_data = test_data.apply(one_hot_hashtags, axis=1)
test_data.head()

,id_original,keyword_original,location_original,text_original,special_chars_count,hashtags,labels,hashtags_count,labels_count,num_chars_count,...,Chicago,Earth,Seattle,"London, UK",Texas,New York City,ss,"Atlanta, GA","Sacramento, CA","London, England"
0,0,NaN,NaN,just happened terrible car crash,0,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,NaN,NaN,heard about earthquake is different cities sta...,3,#earthquake,NaN,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,NaN,NaN,there is forest fire at spot pond geese are f...,2,NaN,NaN,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,NaN,NaN,apocalypse lighting spokane wildfires,3,#spokane #wildfires,NaN,2,0,0,...,0,0,0,0,0,0,0,0,0,0
4,11,NaN,NaN,typhoon soudelor kills in china and taiwan,0,NaN,NaN,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [26]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text  import TfidfVectorizer
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder

In [27]:
train, train_test = train_test_split(train_data, test_size=0.5)

In [28]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
                    ])

In [29]:
text_clf = text_clf.fit(train.clean_text, train.target_label)

In [30]:
train_predicted = text_clf.predict(train_test.clean_text)
train_predicted_proba = text_clf.predict_proba(train_test.clean_text)
np.mean(train_predicted == train_test.target_label)

0.7938008930916732

In [31]:
test_predicted = text_clf.predict(test_data.clean_text)
test_predicted_proba = text_clf.predict_proba(test_data.clean_text)

In [32]:
train_test['NB_target_proba_not'] = train_predicted_proba.T[0]
test_data['NB_target_proba_not'] = test_predicted_proba.T[0]

In [33]:
text_clf_svm = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf-svm', SGDClassifier(loss='log', penalty='l2',
                                                   alpha=1e-3, random_state=42)),
                        ])

In [34]:
text_clf_svm = text_clf_svm.fit(train.clean_text, train.target_label)

In [35]:
train_predicted = text_clf_svm.predict(train_test.clean_text)
train_predicted_proba = text_clf_svm.predict_proba(train_test.clean_text)
np.mean(train_predicted == train_test.target_label)

0.774100341476228

In [36]:
test_predicted = text_clf_svm.predict(test_data.clean_text)
test_predicted_proba = text_clf_svm.predict_proba(test_data.clean_text)

In [37]:
train_test['SVM_target_proba_not'] = train_predicted_proba.T[0]
test_data['SVM_target_proba_not'] = test_predicted_proba.T[0]

In [38]:
train_test.drop(["id_original","keyword_original","location_original","text_original","hashtags","labels","clean_text"], axis=1, inplace=True)
real_test_data = test_data.drop(["id_original","keyword_original","location_original","text_original","hashtags","labels","clean_text"], axis=1)

In [39]:
train, test = train_test_split(train_test, test_size=0.2)

In [40]:
model_xgb = xgb.XGBClassifier(n_estimators=300, colsample_bytree=0.5, learning_rate=0.1, max_depth=11)
model_xgb.fit(train.drop(['target_label'], axis=1), train.target_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [41]:
test_prediction = model_xgb.predict(test.drop(['target_label'], axis=1))
print("Accuracy score: %f" % (accuracy_score(test.target_label, test_prediction)))

Accuracy score: 0.787402


In [42]:
pd.DataFrame(model_xgb.feature_importances_, index=train.drop(['target_label'], axis=1).columns, columns=["importancia"]).\
        sort_values(by="importancia",ascending=False)

,importancia
SVM_target_proba_not,0.18
NB_target_proba_not,0.09
links_count,0.08
labels_count,0.06
Florida,0.06
mean_word_length,0.06
text_length,0.05
vowels_count,0.05
special_chars_count,0.05
num_chars_count,0.05


In [26]:
model_xgb = xgb.XGBClassifier(n_estimators=300, colsample_bytree=0.5, learning_rate=0.1, max_depth=11)
model_xgb.fit(train_test.drop(['target_label'], axis=1), train_test.target_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.5, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.1, max_delta_step=0, max_depth=11,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=300, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
real_test_prediction = model_xgb.predict(real_test_data)
test_data['target'] = real_test_prediction
test_data[['id_original', 'target']].rename(columns={'id_original': 'id'}).to_csv('~/Documents/Datos/DataSets/TP2/res_XGB_1.csv', index=False)